In [108]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [156]:
excel_file_path = 'all_data.xlsx'
csv_file_path = 'all_data.csv'
df = pd.read_excel(excel_file_path, sheet_name=0)
# 削除する列のリスト
columns_to_drop = ['Density [kg/m^3]', '緯度', '経度', 'DO eq', '備考', '水深', 'pH', 'Pressure', 'O2 weiss 1970','DIC[μmol/L@25℃]','Depth[m]', 'DO[ µmol/kg]', 'TA [μmol/kg]','Salinity']
df.drop(columns=columns_to_drop, inplace=True)
target_places = ['苫小牧', 'オーストラリア北東', 'インドネシア', 'オーストラリア南東',
                 'マダガスカル', 'ギアナ沖', '北大西洋']

df = df[
    ~((df['場所'].isin(target_places)) & (df['label'] == -1))
]
# CSVファイルとして保存
df.to_csv(csv_file_path, index=False)
print(f"CSVファイルが保存されました: {csv_file_path}")
df

CSVファイルが保存されました: all_data.csv


,No.,pCO2,AOU,excess DIC,T,nDIC+0.768DO,DIC-0.5TA+0.83DO,場所,日時,DO Saturation,label
0,1,287.110917,-59.375370,-76.860917,27.8250,2114.488440,944.092151,橘湾,2020-08-31,1.295823,-1
1,2,425.777660,12.204354,2.096881,25.6610,2166.235913,985.000393,橘湾,2020-08-31,0.941249,-1
2,3,420.944503,14.201825,-0.024379,25.3680,2164.846536,984.128603,橘湾,2020-08-31,0.931941,-1
3,4,487.481913,72.631801,26.032198,22.6560,2172.105752,989.467402,橘湾,2020-08-31,0.666868,-1
4,5,466.192694,23.793279,18.832014,25.5250,2175.477671,997.066525,橘湾,2020-08-31,0.885662,-1
...,...,...,...,...,...,...,...,...,...,...,...
2060,2061,542.771000,118.587515,49.811000,10.6016,2265.132400,1143.534000,北大西洋,NaT,0.521004,0
2061,2062,557.977000,130.831224,53.484000,10.1420,2265.232200,1141.832000,北大西洋,NaT,0.483192,0
2062,2063,519.493000,119.491733,43.128000,10.7688,2255.845200,1134.107000,北大西洋,NaT,0.516312,0
2063,2064,538.981000,128.104189,48.153000,10.0288,2263.360400,1140.654000,北大西洋,NaT,0.493668,0


In [151]:
def analyze_with_prediction_interval(df, param1, param2, place, place_name, figsize=(12, 8)):# 線形回帰の場合
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    #データを準備
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)

    # 回帰直線の予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    y_pred = slope * x_line.flatten() + intercept

    # 予測区間の計算
    # 信頼度95%の予測区間を計算
    pi = 0.95

    # 予測区間の計算に必要な統計量
    n = len(x)  # label = 1 のデータ数
    x_mean = np.mean(x)

    # 標準誤差の計算
    sum_sq_err = np.sum((y - (slope * x.flatten() + intercept))**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # 予測区間の計算
    x_new = x_line.flatten()

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 予測区間の幅を計算
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (x_new - np.mean(x.flatten()))**2 /
                                    np.sum((x.flatten() - np.mean(x.flatten()))**2))

    # 上側予測区間
    upper_pi = y_pred + pi_width

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point = df[df['場所']==place]
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               label=place_name,
               alpha=1,
               s=50)

    # 回帰直線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Regression Line (y = {slope:.3f}x + {intercept:.3f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in {place} with Regression Line: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'regression_analysis_{place}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、回帰線からの予測値を計算

    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = slope * x_all.flatten() + intercept

    # 実際の値と予測値の差を計算し、回帰線より上のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y > y_pred_data].copy()
    outliers_seepage = outliers[outliers['場所']==place]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_seepage}")
    return outliers

In [152]:
def analyze_with_prediction_interval_pCO2_DOsaturation(df, param1, param2, place, place_name, figsize=(12, 8)): #累乗近似の場合
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 対数変換
    log_x = np.log(x)
    log_y = np.log(y)

    # 対数空間で線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_x.flatten(), log_y)

    # 累乗関数のパラメータに変換
    a = np.exp(intercept)  # 係数
    b = slope  # 指数

    # 予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    # 累乗関数での予測値
    y_pred = a * (x_line ** b).flatten()

    # 予測区間の計算（対数空間で）
    pi = 0.95
    n = len(x)
    log_x_mean = np.mean(log_x)

    # 対数空間での標準誤差
    log_y_pred = slope * log_x.flatten() + intercept
    sum_sq_err = np.sum((log_y - log_y_pred)**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 対数空間での予測区間
    log_x_new = np.log(x_line.flatten())
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (log_x_new - log_x_mean)**2 /
                                    np.sum((log_x.flatten() - log_x_mean)**2))

    # 予測区間を元のスケールに戻す
    log_y_pred_line = slope * log_x_new + intercept
    upper_pi = np.exp(log_y_pred_line + pi_width)

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point = df[df['場所']==place]
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               label=place_name,
               alpha=1,
               s=50)


    # 累乗回帰曲線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Power Regression (y = {a:.2f}x^{b:.2f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in {place_name} with Power Regression: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'power_regression_analysis_{place}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、累乗回帰での予測値を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = a * (x_all ** b).flatten()

    # 実際の値と予測値の差を計算し、回帰線より上のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y > y_pred_data].copy()
    outliers_seepage = outliers[outliers['場所']==place]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_seepage}")

    return outliers

In [171]:
def analyze_with_prediction_interval(df, param1, param2, unit, place, place_name, indicator, figsize=(16, 8)):
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    #データを準備
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)

    # 回帰直線の予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    y_pred = slope * x_line.flatten() + intercept

    # 予測区間の計算
    # 信頼度95%の予測区間を計算
    pi = 0.95

    # 予測区間の計算に必要な統計量
    n = len(x)  # label = 1 のデータ数
    x_mean = np.mean(x)

    # 標準誤差の計算
    sum_sq_err = np.sum((y - (slope * x.flatten() + intercept))**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # 予測区間の計算
    x_new = x_line.flatten()

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 予測区間の幅を計算
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (x_new - np.mean(x.flatten()))**2 /
                                    np.sum((x.flatten() - np.mean(x.flatten()))**2))

    # 上側予測区間
    upper_pi = y_pred + pi_width

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("white")

    # 実データのプロット
    data_point = df[df['場所']==place]
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               label=place_name,
               alpha=1,
               s=50)

    # 回帰直線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Regression Line (y = {slope:.3f}x + {intercept:.3f})')
    #plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')
    #卒論用にコメントアウト0104mtgに元画像あり

    # グラフの設定
    plt.xlabel(f'{param2}({unit})', fontsize=16)
    plt.ylabel(f'{param1}(μmol/kg)', fontsize=16)
    # plt.title(f'Data Distribution in {place_name} with Regression Line: {param1} vs {param2}', fontsize=14)
    plt.title(f'Data distirbution of {place_name} by {indicator}', fontsize=20)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)

    plt.tight_layout()
    plt.savefig(f'regression_analysis_{place}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、予測区間を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = slope * x_all.flatten() + intercept

    # 全データポイントの予測区間幅を計算
    pi_width_data = t * std_err * np.sqrt(1 + 1/n +
                                         (x_all.flatten() - np.mean(x.flatten()))**2 /
                                         np.sum((x.flatten() - np.mean(x.flatten()))**2))

    # 上側予測区間を計算
    upper_pi_data = y_pred_data + pi_width_data

    # 実際の値と予測区間の比較
    actual_y = df[param1].values

    # 回帰線より上のデータポイントを特定
    outliers_regression = df[actual_y > y_pred_data].copy()
    outliers_regression_seepage = outliers_regression[outliers_regression['場所']==place]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_regression_seepage}")

    # 95%上側予測区間より上のデータポイントを特定
    outliers_pi = df[actual_y > upper_pi_data].copy()
    outliers_pi_seepage = outliers_pi[outliers_pi['場所']==place]['No.'].tolist()
    print(f"  - 95%上側予測区間より上にあるデータ: No.{outliers_pi_seepage}")

    return outliers_regression, outliers_pi

def analyze_with_prediction_interval_pCO2_DOsaturation(df, param1, param2, place, place_name, figsize=(16, 8)):
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 対数変換
    log_x = np.log(x)
    log_y = np.log(y)

    # 対数空間で線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_x.flatten(), log_y)

    # 累乗関数のパラメータに変換
    a = np.exp(intercept)  # 係数
    b = slope  # 指数

    # 予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    # 累乗関数での予測値
    y_pred = a * (x_line ** b).flatten()

    # 予測区間の計算（対数空間で）
    pi = 0.95
    n = len(x)
    log_x_mean = np.mean(log_x)

    # 対数空間での標準誤差
    log_y_pred = slope * log_x.flatten() + intercept
    sum_sq_err = np.sum((log_y - log_y_pred)**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 対数空間での予測区間
    log_x_new = np.log(x_line.flatten())
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (log_x_new - log_x_mean)**2 /
                                    np.sum((log_x.flatten() - log_x_mean)**2))

    # 予測区間を元のスケールに戻す
    log_y_pred_line = slope * log_x_new + intercept
    upper_pi = np.exp(log_y_pred_line + pi_width)

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("white")

    # 実データのプロット
    data_point = df[df['場所']==place]
    plt.scatter(data_point[param2],
               data_point[param1],
               color='black',
               marker='o',
               label=place_name,
               alpha=1,
               s=50)

    # 累乗回帰曲線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Power Regression (y = {a:.2f}x^{b:.2f})')
    # plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')
    #卒論用にコメントアウト0104mtgに元画像あり

    plt.ylim([0, 1000])
    plt.xlim([0.6, 1.3])

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=16)
    plt.ylabel(f'{param1}(μatm)', fontsize=16)
    #plt.title(f'Data Distribution in {place_name} with Power Regression: {param1} vs {param2}', fontsize=14)
    plt.title(f'Data distribution of {place_name} by PC-OS', fontsize=20)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)

    plt.tight_layout()
    plt.savefig(f'power_regression_analysis_{place}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、累乗回帰での予測値を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = a * (x_all ** b).flatten()

    # 全データポイントの予測区間を計算（対数空間で）
    log_x_all = np.log(x_all.flatten())
    pi_width_data = t * std_err * np.sqrt(1 + 1/n +
                                         (log_x_all - log_x_mean)**2 /
                                         np.sum((log_x.flatten() - log_x_mean)**2))

    # 予測区間を元のスケールに戻す
    log_y_pred_data = slope * log_x_all + intercept
    upper_pi_data = np.exp(log_y_pred_data + pi_width_data)

    # 実際の値と予測値の差を計算
    actual_y = df[param1].values

    # 回帰線より上のデータポイントを特定
    outliers_regression = df[actual_y > y_pred_data].copy()
    outliers_regression_seepage = outliers_regression[outliers_regression['場所']==place]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_regression_seepage}")

    # 95%上側予測区間より上のデータポイントを特定
    outliers_pi = df[actual_y > upper_pi_data].copy()
    outliers_pi_seepage = outliers_pi[outliers_pi['場所']==place]['No.'].tolist()
    print(f"  - 95%上側予測区間より上にあるデータ: No.{outliers_pi_seepage}")

    return outliers_regression, outliers_pi

def make_analysis(place, place_name):
    outliers1 = analyze_with_prediction_interval_pCO2_DOsaturation(df, 'pCO2', 'DO Saturation', place, place_name)
    outliers2 = analyze_with_prediction_interval(df, 'excess DIC', 'AOU','μmol/kg', place, place_name, 'EC-AO')
    outliers3 = analyze_with_prediction_interval(df, 'nDIC+0.768DO', 'T','°C', place, place_name, 'NCO-T')
    outliers4 = analyze_with_prediction_interval(df, 'DIC-0.5TA+0.83DO', 'T','°C', place, place_name,'CAO-T')

In [172]:
make_analysis('橘湾温泉', 'Tachibana')

  - 回帰線より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156]
  - 95%上側予測区間より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
  - 回帰線より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
  - 95%上側予測区間より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153]
  - 回帰線より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156]
  - 95%上側予測区間より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
  - 回帰線より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154]
  - 95%上側予測区間より上にあるデータ: No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153

In [173]:
make_analysis('舘山', 'Tateyama')

  - 回帰線より上にあるデータ: No.[159, 160, 161, 162, 163, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 184, 209, 213, 214, 215, 216, 217, 218, 220, 221, 222, 225, 227, 228, 229, 230]
  - 95%上側予測区間より上にあるデータ: No.[159, 160, 161, 162, 213, 214]
  - 回帰線より上にあるデータ: No.[159, 160, 161, 162, 163, 166, 213, 214, 215, 230]
  - 95%上側予測区間より上にあるデータ: No.[159, 160, 161, 162, 214]
  - 回帰線より上にあるデータ: No.[159, 160, 161, 162, 163, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 191, 192, 194, 205, 206, 209, 210, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 227, 228, 229, 230, 231]
  - 95%上側予測区間より上にあるデータ: No.[159, 160, 161, 162, 213, 214]
  - 回帰線より上にあるデータ: No.[159, 160, 161, 162, 163, 213, 214, 215, 217, 230]
  - 95%上側予測区間より上にあるデータ: No.[159, 160, 161, 162]


In [116]:
#二つの場所を同時にグラフにする

def plot_with_prediction_interval(df, param1, param2, place1, place_name1, place2, place_name2, figsize=(12, 8)):# 線形回帰の場合
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東', 'マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    #データを準備
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)

    # 回帰直線の予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    y_pred = slope * x_line.flatten() + intercept

    # 予測区間の計算
    # 信頼度95%の予測区間を計算
    pi = 0.95

    # 予測区間の計算に必要な統計量
    n = len(x)  # label = 1 のデータ数
    x_mean = np.mean(x)

    # 標準誤差の計算
    sum_sq_err = np.sum((y - (slope * x.flatten() + intercept))**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # 予測区間の計算
    x_new = x_line.flatten()

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 予測区間の幅を計算
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (x_new - np.mean(x.flatten()))**2 /
                                    np.sum((x.flatten() - np.mean(x.flatten()))**2))

    # 上側予測区間
    upper_pi = y_pred + pi_width

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point1 = df[df['場所']==place1]
    plt.scatter(data_point1[param2],
               data_point1[param1],
               color='black',
               marker='o',
               label=place_name1,
               alpha=1,
               s=50)

    # 実データのプロット
    data_point2 = df[df['場所']==place2]
    plt.scatter(data_point2[param2],
               data_point2[param1],
               color='black',
               marker='o',
               label=place_name2,
               alpha=1,
               s=50)

    # 回帰直線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Regression Line (y = {slope:.3f}x + {intercept:.3f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in {place_name1} & {place_name2} with Regression Line: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'regression_analysis_{place1}&{place2}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    """
    # 全データポイントについて、回帰線からの予測値を計算

    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = slope * x_all.flatten() + intercept

    # 実際の値と予測値の差を計算し、回帰線より上のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y > y_pred_data].copy()
    outliers_seepage = outliers[outliers['場所']==place1]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_seepage}")
    return outliers
    """

def plot_with_prediction_interval_pCO2_DOsaturation(df, param1, param2, place1, place_name1, place2, place_name2, figsize=(12, 8)): #累乗近似の場合
    # 苫小牧とNOAAのデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東', 'マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 対数変換
    log_x = np.log(x)
    log_y = np.log(y)

    # 対数空間で線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_x.flatten(), log_y)

    # 累乗関数のパラメータに変換
    a = np.exp(intercept)  # 係数
    b = slope  # 指数

    # 予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    # 累乗関数での予測値
    y_pred = a * (x_line ** b).flatten()

    # 予測区間の計算（対数空間で）
    pi = 0.95
    n = len(x)
    log_x_mean = np.mean(log_x)

    # 対数空間での標準誤差
    log_y_pred = slope * log_x.flatten() + intercept
    sum_sq_err = np.sum((log_y - log_y_pred)**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 対数空間での予測区間
    log_x_new = np.log(x_line.flatten())
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (log_x_new - log_x_mean)**2 /
                                    np.sum((log_x.flatten() - log_x_mean)**2))

    # 予測区間を元のスケールに戻す
    log_y_pred_line = slope * log_x_new + intercept
    upper_pi = np.exp(log_y_pred_line + pi_width)

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")

    # 実データのプロット
    data_point1 = df[df['場所']==place1]
    plt.scatter(data_point1[param2],
               data_point1[param1],
               color='black',
               marker='o',
               label=place_name1,
               alpha=1,
               s=50)

    data_point2 = df[df['場所']==place2]
    plt.scatter(data_point2[param2],
               data_point2[param1],
               color='black',
               marker='o',
               label=place_name2,
               alpha=1,
               s=50)


    # 累乗回帰曲線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Power Regression (y = {a:.2f}x^{b:.2f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=12)
    plt.ylabel(f'{param1}', fontsize=12)
    plt.title(f'Data Distribution in {place_name1} & {place_name2} with Power Regression: {param1} vs {param2}', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(f'power_regression_analysis_{place1} & {place2}_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 全データポイントについて、累乗回帰での予測値を計算
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = a * (x_all ** b).flatten()

    """
    # 実際の値と予測値の差を計算し、回帰線より上のポイントを特定
    actual_y = df[param1].values
    outliers = df[actual_y > y_pred_data].copy()
    outliers_seepage = outliers[outliers['場所']==place]['No.'].tolist()
    print(f"  - 回帰線より上にあるデータ: No.{outliers_seepage}")

    return outliers
    """

In [117]:
plot_with_prediction_interval_pCO2_DOsaturation(df, 'pCO2 [μatm]', 'DO Saturation', '橘湾', 'Tachibana', '舘山', 'Tateyama')
plot_with_prediction_interval(df, 'excess DIC', 'AOU', '橘湾', 'Tachibana', '舘山', 'Tateyama')
plot_with_prediction_interval(df, 'nDIC+0.768DO', 'T', '橘湾', 'Tachibana', '舘山', 'Tateyama')
plot_with_prediction_interval(df, 'DIC-0.5TA+0.83DO', 'T', '橘湾', 'Tachibana', '舘山', 'Tateyama')